In [1]:
from google.cloud import bigquery
import pandas as pd
import numpy as np
import seaborn as sns
#import pandas_gbp
import matplotlib.pyplot as plt
%matplotlib inline
from datetime import datetime
from matplotlib.ticker import (AutoMinorLocator, MultipleLocator)

class Covid19:
    df_cumulative_raw = pd.DataFrame()
    df_cumulative_by_country = pd.DataFrame()
    df_daily = pd.DataFrame()
    df_world_population = pd.DataFrame()
    df_confirmed = pd.DataFrame()
    df_deaths = pd.DataFrame()
    df_recovered = pd.DataFrame()
    
    COL_COUNTRY = "country"
    COL_DATE = "date"
    COL_CONFIRMED = "confirmed"
    COL_DEATHS = "deaths"
    COL_RECOVERED = "recovered"
    COL_PROVINCE = "province_state"
    COL_ACTIVE = 'active'
    COL_POPULATION = 'population'
    COL_DAILY_CASES = 'Daily Cases'
    COL_DAILY_RECOVERED = 'Daily Recovered'
    COL_DAILY_DEATHS = 'Daily Deaths'
    SUFFIX_100K = '_per100k'
    SUFFIX_1M = '_per1m'

    def __init__(self, datefilter: str = '2020-01-01'):
        self.data = []
        self.df_confirmed = self.fetch_covid19_cases('confirmed_cases') #this is a tablename
        self.df_deaths = self.fetch_covid19_cases(self.COL_DEATHS)
        self.df_recovered = self.fetch_covid19_cases('recovered_cases') #this is a tablename
        self.df_world_population = self.fetch_population()
        self.preprocess(datefilter)
        #self.calculate_new()

    def preprocess(self, datefilter: str):
        #Preprocess
        #remove canada as per https://towardsdatascience.com/covid-19-data-processing-58aaa3663f6
        self.df_recovered = self.df_recovered[self.df_recovered[self.COL_COUNTRY]!='Canada']
        #join the confirmed, deaths and recovered data sets
        df_covid = self.covid_flip_datetocolumn(self.df_confirmed, self.COL_CONFIRMED).merge(
            right = self.covid_flip_datetocolumn(self.df_deaths, self.COL_DEATHS), 
            how='left',
            on=[self.COL_COUNTRY, self.COL_PROVINCE, self.COL_DATE]
        ).merge(
            right = self.covid_flip_datetocolumn(self.df_recovered, self.COL_RECOVERED),
            how='left',
            on=[self.COL_COUNTRY, self.COL_PROVINCE, self.COL_DATE]
        )
        #remove ship data  https://towardsdatascience.com/covid-19-data-processing-58aaa3663f6
        ship_rows = df_covid[self.COL_COUNTRY].str.contains('Diamond Princess') | df_covid[self.COL_COUNTRY].str.contains('MS Zaandam')
        nan_rows = df_covid[self.COL_PROVINCE].isna()
        df_covid = df_covid[~(ship_rows)]
        df_covid = df_covid[~(nan_rows)]
        #Calculate active cases
        df_covid[self.COL_ACTIVE] = df_covid[self.COL_CONFIRMED] - df_covid[self.COL_DEATHS] - df_covid[self.COL_RECOVERED]
        #Join with world population
        df_covid_popl = df_covid.join(self.df_world_population, how='outer', on= self.COL_COUNTRY)
        #China doesn't have a summary row (province = null) so calculate
        df_covid_popl = self.__fix_countries_no_summary(df_covid_popl, 'China')

        #TODO: This should probably be only be done for the by_country dataframe not raw
        for var in [self.COL_CONFIRMED, self.COL_DEATHS, self.COL_RECOVERED, self.COL_ACTIVE]:
            df_covid_popl[var + self.SUFFIX_1M] = df_covid_popl[var] * 1000000 / df_covid_popl[self.COL_POPULATION]
            df_covid_popl[var + self.SUFFIX_100K] = df_covid_popl[var] * 100000 / df_covid_popl[self.COL_POPULATION]

        #df_covid_popl.dropna(subset=[self.COL_DATE], inplace=True)
        df_covid_popl = df_covid_popl[df_covid_popl[self.COL_DATE] > np.datetime64(datefilter)]

        self.df_cumulative_raw = df_covid_popl
        self.df_cumulative_by_country = self.df_cumulative_raw[self.df_cumulative_raw[self.COL_PROVINCE] == 'ALL']
        self.df_cumulative_by_country.drop([self.COL_PROVINCE], axis=1, inplace=True)


        #calculate daily numbers
        self.calculate_daily_nos()

    def fetch_covid19_cases(self, table):
        # Construct a BigQuery client object.
        client = bigquery.Client()
        query = """
            SELECT *, 
            FROM `bigquery-public-data.covid19_jhu_csse.{0}`
        """.format(table)
        df_confirmed_cases = client.query(query).to_dataframe()
        df_confirmed_cases.drop(['latitude', 'longitude', 'location_geom'], axis=1, inplace=True)
        df_confirmed_cases['province_state'] = df_confirmed_cases['province_state'].fillna('ALL')
        df_confirmed_cases.rename(columns={'country_region': self.COL_COUNTRY}, inplace=True)
        return df_confirmed_cases
    
    country_data = pd.DataFrame()
    def __fix_countries_no_summary(self, df_covid_popl: pd.DataFrame, country: str):
        groupbycountry = df_covid_popl[
                            df_covid_popl[self.COL_COUNTRY] == country
                        ].groupby([self.COL_COUNTRY, self.COL_DATE])
        country_data = groupbycountry[
                            self.COL_CONFIRMED, self.COL_DEATHS, self.COL_RECOVERED, self.COL_ACTIVE
                        ].sum()
        country_data = country_data.join(groupbycountry[self.COL_POPULATION,'country_code'].first())
        country_data.reset_index(inplace=True)
        country_data[self.COL_PROVINCE] = 'ALL'
        self.country_data = country_data
        return df_covid_popl.append(country_data)

    def fetch_population(self):
        client = bigquery.Client()
        query = """
            SELECT country_code, country_name as country, midyear_population as population 
            FROM `bigquery-public-data.census_bureau_international.midyear_population`
            where year = EXTRACT(year from (CURRENT_DATE()))
            order by midyear_population desc
        """
        df_world_population = client.query(query).to_dataframe()
        df_world_population[self.COL_COUNTRY][df_world_population[self.COL_COUNTRY] == 'United States'] = 'US'
        df_world_population = df_world_population.set_index(self.COL_COUNTRY)
        return df_world_population

    def covid_flip_datetocolumn(self, df_covid: pd.DataFrame, col_name: str):
        dates = df_covid.columns[2:] #excluding country and province
        df_temp = df_covid.melt(
            id_vars = [self.COL_COUNTRY, self.COL_PROVINCE], 
            value_vars = dates,
            var_name = self.COL_DATE,
            value_name = col_name
        )
        df_temp[self.COL_DATE] = df_temp[self.COL_DATE].map(lambda strtodate: strtodate.replace('_','/')[1:])
        df_temp[self.COL_DATE] = df_temp[self.COL_DATE].map(lambda x: datetime.strptime(x, "%m/%d/%y"))
        return df_temp
        
    # Calculates ratio of two countries numbes either confirmed cases, deaths, recovered
    def covid_ratio_by_country(self, col_name, source, dest):
        df_top = self.df_cumulative_by_country[[col_name, self.COL_DATE]][self.df_cumulative_by_country[self.COL_COUNTRY] == source] 
        df_down = self.df_cumulative_by_country[[col_name, self.COL_DATE]][self.df_cumulative_by_country[self.COL_COUNTRY] == dest] 
        df_ratio = df_top.merge(df_down, on='date')
        df_ratio['ratio'] = df_ratio[col_name + '_x'] / df_ratio[col_name + '_y'] 
        return df_ratio[[self.COL_DATE, 'ratio']]

    def calculate_daily_nos(self):
        # new cases 
        df_covid_popl_new = self.df_cumulative_by_country.groupby(
            [self.COL_DATE, self.COL_COUNTRY])[
                self.COL_CONFIRMED, self.COL_DEATHS, self.COL_RECOVERED, self.COL_ACTIVE, self.COL_POPULATION,
                self.COL_CONFIRMED + self.SUFFIX_1M, self.COL_DEATHS + self.SUFFIX_1M, self.COL_RECOVERED + self.SUFFIX_1M,
                self.COL_CONFIRMED + self.SUFFIX_100K, self.COL_DEATHS + self.SUFFIX_100K, self.COL_RECOVERED + self.SUFFIX_100K,
            ].sum().reset_index()
        temp = df_covid_popl_new.groupby([self.COL_COUNTRY, self.COL_DATE])[self.COL_CONFIRMED, self.COL_DEATHS, self.COL_RECOVERED]
        temp = temp.sum().diff().reset_index()
        mask = temp[self.COL_COUNTRY] != temp[self.COL_COUNTRY].shift(1)
        temp.loc[mask, self.COL_CONFIRMED] = np.nan
        temp.loc[mask, self.COL_DEATHS] = np.nan
        temp.loc[mask, self.COL_RECOVERED] = np.nan
        # renaming columns
        temp.columns = [self.COL_COUNTRY, self.COL_DATE, self.COL_DAILY_CASES, self.COL_DAILY_DEATHS, self.COL_DAILY_RECOVERED]
        # merging new values
        df_covid_popl_new = pd.merge(df_covid_popl_new, temp, on=[self.COL_COUNTRY, self.COL_DATE])
        # filling na with 0
        df_covid_popl_new = df_covid_popl_new.fillna(0)
        # fixing data types
        cols = [self.COL_DAILY_CASES, self.COL_DAILY_DEATHS, self.COL_DAILY_RECOVERED]
        df_covid_popl_new[cols] = df_covid_popl_new[cols].astype('int')
        # 
        df_covid_popl_new[self.COL_DAILY_CASES] = df_covid_popl_new[self.COL_DAILY_CASES].apply(lambda x: 0 if x<0 else x)
        for var in [self.COL_DAILY_CASES, self.COL_DAILY_DEATHS, self.COL_DAILY_RECOVERED]:
            df_covid_popl_new[var + '_per1k'] = df_covid_popl_new[var] * 1000 / df_covid_popl_new[self.COL_POPULATION]
        self.df_daily = df_covid_popl_new


In [30]:
covid19 = Covid19()


In [31]:
covid19.df_cumulative_raw[covid19.df_cumulative_raw['country'] =='China']['province_state'].unique()

array(['Tibet', 'Macau', 'Qinghai', 'Sichuan', 'Fujian', 'Hebei',
       'Chongqing', 'Guangdong', 'Ningxia', 'Inner Mongolia', 'Xinjiang',
       'Shanghai', 'Gansu', 'Jilin', 'Hong Kong', 'Jiangsu', 'Liaoning',
       'Shanxi', 'Tianjin', 'Hubei', 'Guizhou', 'Beijing', 'Jiangxi',
       'Hainan', 'Yunnan', 'Zhejiang', 'Anhui', 'Heilongjiang', 'Shaanxi',
       'Shandong', 'Henan', 'Hunan', 'Guangxi', 'ALL'], dtype=object)

In [32]:
covid19.df_cumulative_by_country[covid19.df_cumulative_by_country['country'] =='China']

,country,date,confirmed,deaths,recovered,active,country_code,population,confirmed_per1m,confirmed_per100k,deaths_per1m,deaths_per100k,recovered_per1m,recovered_per100k,active_per1m,active_per100k
30.0,China,2020-02-21,75550.0,2238.0,18704.0,54608.0,CH,1.394016e+09,54.195936,5.419594,1.605434,0.160543,13.417350,1.341735,39.173152,3.917315
31.0,China,2020-02-22,77001.0,2443.0,22699.0,51859.0,CH,1.394016e+09,55.236813,5.523681,1.752491,0.175249,16.283171,1.628317,37.201152,3.720115
32.0,China,2020-02-23,77022.0,2445.0,23187.0,51390.0,CH,1.394016e+09,55.251878,5.525188,1.753925,0.175393,16.633238,1.663324,36.864714,3.686471
33.0,China,2020-02-24,77241.0,2595.0,25015.0,49631.0,CH,1.394016e+09,55.408978,5.540898,1.861528,0.186153,17.944558,1.794456,35.602892,3.560289
34.0,China,2020-02-25,77754.0,2665.0,27676.0,47413.0,CH,1.394016e+09,55.776979,5.577698,1.911743,0.191174,19.853431,1.985343,34.011805,3.401181
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
133.0,China,2020-06-03,84160.0,4638.0,79404.0,118.0,CH,1.394016e+09,60.372335,6.037234,3.327078,0.332708,56.960610,5.696061,0.084648,0.008465
134.0,China,2020-06-04,84171.0,4638.0,79415.0,118.0,CH,1.394016e+09,60.380226,6.038023,3.327078,0.332708,56.968501,5.696850,0.084648,0.008465
135.0,China,2020-06-05,84177.0,4638.0,79420.0,119.0,CH,1.394016e+09,60.384530,6.038453,3.327078,0.332708,56.972087,5.697209,0.085365,0.008536
136.0,China,2020-06-06,84186.0,4638.0,79424.0,124.0,CH,1.394016e+09,60.390986,6.039099,3.327078,0.332708,56.974957,5.697496,0.088952,0.008895


ValueError: cannot insert level_0, already exists